### Librerias

In [108]:
import pandas as pd
from sklearn import preprocessing

### Cargar datos en bruto

In [4]:
aprobados = pd.read_csv('data/aprobados.csv')
negados = pd.read_csv('data/negados.csv')

### Eliminar columnas que no son necesarias

In [118]:
eliminar = ['Unnamed: 0', 'Unnamed: 0.1', 'Total activos', 'ConoCliente', 'Fecha_corte', 'Ocupacion', 'Agencia', 'Cargo', 'Ciiu', 'Estado', 'EndeuIndi', 'Califica', 'FecIngEmp', 'FecUltAct', 'CuidadRes']

In [119]:
def eliminar_columnas(aprobados, negados, eliminar):
    aprob = aprobados.copy()
    neg = negados.copy()
    for ii in eliminar:
        aprob.pop(ii)
        neg.pop(ii)
    return aprob, neg

In [120]:
aprob, neg = eliminar_columnas(aprobados, negados, eliminar)

In [121]:
aprob.head(3)

,Edad,Sexo,Perscargo,Estrato,Nivel de estudios,Tipo de contrato,Ingresos,Egresos,Tipo de actividad,Tipo de vivienda,...,Destino,Monto,Plazo,Cuota,Garantia,RespPat,FecIngreso,Pagare,MoraCome,Puntaje
0,22,no,0,2,Técnico,Indefinido,949000,226000,Empleado,Familiar,...,Consumo,1600000,24,40930,Libranza Establecida,Sin Patrimonio,2007-03-29,41707.0,Con Experiencia sin Mora,27.0
1,57,no,0,4,Bachiller,Ninguno,1806000,1319000,Jubilado,Propia,...,Consumo,5000000,36,213.932,Sin Codeudor,Prop.sin Hipoteca,2001-02-26,NaN,Con Experiencia sin Mora,63.0
2,21,no,0,2,Bachiller,Fijo,816000,157000,Empleado,Familiar,...,Consumo,6200000,36,247167,Con Codeudor(es),Sin Patrimonio,2006-10-05,41617.0,Con Experiencia sin Mora,NaN


### Configurar columnas con variables tipo fecha

In [86]:
date_today = '20200815'
aprob['FecIngreso'] = pd.to_datetime(aprob['FecIngreso'])
neg['FecIngreso'] = pd.to_datetime(neg['FecIngreso'])
aprob['NumDias'] = (pd.to_datetime(date_today, format='%Y%m%d', errors='ignore') - pd.to_datetime(aprob['FecIngreso'])).dt.days
neg['NumDias'] = (pd.to_datetime(date_today, format='%Y%m%d', errors='ignore') - pd.to_datetime(neg['FecIngreso'])).dt.days

In [122]:
aprob.head(3)

,Edad,Sexo,Perscargo,Estrato,Nivel de estudios,Tipo de contrato,Ingresos,Egresos,Tipo de actividad,Tipo de vivienda,...,Destino,Monto,Plazo,Cuota,Garantia,RespPat,FecIngreso,Pagare,MoraCome,Puntaje
0,22,no,0,2,Técnico,Indefinido,949000,226000,Empleado,Familiar,...,Consumo,1600000,24,40930,Libranza Establecida,Sin Patrimonio,2007-03-29,41707.0,Con Experiencia sin Mora,27.0
1,57,no,0,4,Bachiller,Ninguno,1806000,1319000,Jubilado,Propia,...,Consumo,5000000,36,213.932,Sin Codeudor,Prop.sin Hipoteca,2001-02-26,NaN,Con Experiencia sin Mora,63.0
2,21,no,0,2,Bachiller,Fijo,816000,157000,Empleado,Familiar,...,Consumo,6200000,36,247167,Con Codeudor(es),Sin Patrimonio,2006-10-05,41617.0,Con Experiencia sin Mora,NaN


### Obtener nombres de las columnas categoricas

In [99]:
def variables_categoricas(lista_base, lista_remover):
    categoricas = lista_base
    for ii in lista_remover:
        categoricas.remove(ii)
    return categoricas

In [105]:
var_categoricas = variables_categoricas(list(aprob.columns), list(aprob.describe().columns))
var_categoricas.remove('FecIngreso')
var_categoricas

['Sexo',
 'Nivel de estudios',
 'Tipo de contrato',
 'Tipo de actividad',
 'Tipo de vivienda',
 'Destino',
 'Cuota',
 'Garantia',
 'RespPat',
 'MoraCome']

In [106]:
aprob[var_categoricas]

,Sexo,Nivel de estudios,Tipo de contrato,Tipo de actividad,Tipo de vivienda,Destino,Cuota,Garantia,RespPat,MoraCome
0,no,Técnico,Indefinido,Empleado,Familiar,Consumo,40930,Libranza Establecida,Sin Patrimonio,Con Experiencia sin Mora
1,no,Bachiller,Ninguno,Jubilado,Propia,Consumo,213.932,Sin Codeudor,Prop.sin Hipoteca,Con Experiencia sin Mora
2,no,Bachiller,Fijo,Empleado,Familiar,Consumo,247167,Con Codeudor(es),Sin Patrimonio,Con Experiencia sin Mora
3,no,Técnico,Indefinido,Empleado,Familiar,Consumo,123305,Con Codeudor(es),Sin Patrimonio,Sin experiencia
4,no,Técnico,Fijo,Empleado,Familiar,Consumo,87477,Con Codeudor(es),Sin Patrimonio,Con Experiencia sin Mora
...,...,...,...,...,...,...,...,...,...,...
193,no,Bachiller,Ninguno,Independiente,Familiar,Reposicion de Activos Fij,147.592,Sin Codeudor,Sin Patrimonio,Mora mayor a 60 Dìas
194,yes,Bachiller,Ninguno,Independiente,Familiar,Consumo,113025,Con Codeudor(es),Prop.con Hipoteca,Sin experiencia
195,yes,Profesional,Fijo,Empleado,Familiar,Consumo,387071,Sin Codeudor,Sin Patrimonio,Con Experiencia sin Mora
196,yes,Profesional,Indefinido,Empleado,Propia,Consumo,109736,Libranza Establecida,Sin Patrimonio,Con Experiencia sin Mora


# Procesar variables categoricas

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

In [109]:
le = preprocessing.LabelEncoder()

In [111]:
list(aprob['Nivel de estudios'].unique())

['Técnico',
 'Bachiller',
 'Profesional',
 'Especializacion',
 'Tecnólogo',
 'Primaria',
 'Ninguno']

In [112]:
le.fit(list(aprob['Nivel de estudios'].unique()))

LabelEncoder()

In [113]:
le.classes_

array(['Bachiller', 'Especializacion', 'Ninguno', 'Primaria',
       'Profesional', 'Tecnólogo', 'Técnico'], dtype='<U15')

In [116]:
le.transform(list(le.classes_))

array([0, 1, 2, 3, 4, 5, 6])